# Quiz

In [ ]:
from bs4 import BeautifulSoup
from zipfile import ZipFile
import glob # just to check files with .html extension
import os
import pandas as pd
import re

# Gathering (Coleta)

In [ ]:
data_folder = 'data'
html_folder = os.path.join(data_folder, 'rt_html')

if (os.path.exists(html_folder) and (len(glob.glob1(html_folder, '*.html')) == 100)):
    print('All files was already extracted...')
else:
    if(os.path.exists(f'{html_folder}.zip')):
#         Extract all files to the destination folder
        with ZipFile(os.path.join(data_folder, 'rt_html.zip'), 'r') as datazip:
            datazip.extractall(html_folder)
    else:
#         Data zip file was not found
        print(f'Data zip file wasn\'t found at {data_folder}')
        

In [ ]:
# List of dictionaries to build file by file anda later convert to a DataFrame
df_list = []

for movie_html in os.listdir(html_folder):
    with open(os.path.join(html_folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        
        title = soup.title.contents[0][0:-len(' - Rotten Tomatoes')]
#         audience_score = soup.select('.audience-score.meter')[0].find('span').contents[0][:-1]
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
#         num_audience_ratings = soup.find('div', 'audience-panel').find_all('div', 'audience-info')[0].find_all('div')[1].contents[2]
        num_audience_ratings = soup.find('div', class_='audience-info hidden-xs superPageFontColor')
        num_audience_ratings = num_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')
        
        df_list.append({
            'title': title,
            'audience_score': int(audience_score),
            'number_of_audience_ratings': int(num_audience_ratings)
        })
        
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [ ]:
df.info()

## Solution Test

Run the cell below to see if your solution is correct. If an `AssertionError` is thrown, your solution is incorrect. If no error is thrown, your solution is correct.

### Disclaimmer:

> This assertion check throws an error, although this, after check the .PKL file with the solution, I noticed that apparently that the PKL file does not with the correct encode. The generated dataframe over the HTML files have the same content when compared, but with the right characters encode.

In [ ]:
df_solution = pd.read_pickle('df_solution.pkl')
df.sort_values('title', inplace = True)
df.reset_index(inplace = True, drop = True)
df_solution.sort_values('title', inplace = True)
df_solution.reset_index(inplace = True, drop = True)
pd.testing.assert_frame_equal(df, df_solution)